# Import Library

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create SparkSession Object

In [2]:
spark = SparkSession.builder \
                .appName("Zillow-Home_Prediction") \
                .master("local[*]") \
                .config("spark.sql.repl.eagerEval.enabled", True) \
                .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.maxNumOfFields", 100)

# Data Preprocessing

In [3]:
prop_2016 = spark.read.csv("properties_2016.csv", inferSchema=True, header=True)
prop_2017 = spark.read.csv("properties_2017.csv", inferSchema=True, header=True)
train_2016 = spark.read.csv("train_2016_v2.csv", inferSchema=True, header=True)
train_2017 = spark.read.csv("train_2017.csv", inferSchema=True, header=True)

In [4]:
prop_2016.limit(5)

parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
10754147,NULL,NULL,NULL,0.0,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,6037,NULL,NULL,NULL,NULL,NULL,NULL,34144442,-118654084,85768.0,NULL,NULL,NULL,NULL,NULL,010D,269,NULL,6.0378002041E7,37688,3101,NULL,96337,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,9.0,2015,9.0,NULL,NULL,NULL,NULL
10759547,NULL,NULL,NULL,0.0,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,6037,NULL,NULL,NULL,NULL,NULL,NULL,34140430,-118625364,4083.0,NULL,NULL,NULL,NULL,NULL,0109,261,LCA11*,6.0378001011002E7,37688,3101,NULL,96337,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,27516.0,2015,27516.0,NULL,NULL,NULL,NULL
10843547,NULL,NULL,NULL,0.0,0.0,NULL,NULL,NULL,NULL,NULL,73026.0,NULL,NULL,73026,NULL,NULL,6037,NULL,NULL,NULL,NULL,NULL,NULL,33989359,-118394633,63085.0,NULL,NULL,NULL,NULL,NULL,1200,47,LAC2,6.0377030012017E7,51617,3101,NULL,96095,0.0,NULL,NULL,NULL,2,NULL,NULL,NULL,NULL,NULL,650756.0,1413387.0,2015,762631.0,20800.37,NULL,NULL,NULL
10859147,NULL,NULL,NULL,0.0,0.0,3,7,NULL,NULL,NULL,5068.0,NULL,NULL,5068,NULL,NULL,6037,NULL,NULL,NULL,NULL,NULL,NULL,34148863,-118437206,7521.0,NULL,NULL,NULL,NULL,NULL,1200,47,LAC2,6.0371412023001E7,12447,3101,27080,96424,0.0,NULL,NULL,NULL,NULL,NULL,NULL,1948.0,1,NULL,571346.0,1156834.0,2015,585488.0,14557.57,NULL,NULL,NULL
10879947,NULL,NULL,NULL,0.0,0.0,4,NULL,NULL,NULL,NULL,1776.0,NULL,NULL,1776,NULL,NULL,6037,NULL,NULL,NULL,NULL,NULL,NULL,34194168,-118385816,8512.0,NULL,NULL,NULL,NULL,NULL,1210,31,LAM1,6.0371232052003E7,12447,3101,46795,96450,0.0,NULL,NULL,NULL,1,NULL,NULL,1947.0,NULL,NULL,193796.0,433491.0,2015,239695.0,5725.17,NULL,NULL,NULL


In [5]:
prop_2017.limit(5)

parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
10754147,NULL,NULL,NULL,0.0,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,6037,NULL,NULL,NULL,NULL,NULL,NULL,34144442,-118654084,85768.0,NULL,NULL,NULL,NULL,NULL,010D,269,NULL,6.0378002041E7,37688,3101,NULL,96337,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,9.0,2016,9.0,NULL,NULL,NULL,NULL
10759547,NULL,NULL,NULL,0.0,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,6037,NULL,NULL,NULL,NULL,NULL,NULL,34140430,-118625364,4083.0,NULL,NULL,NULL,NULL,NULL,0109,261,LCA11*,6.0378001011002E7,37688,3101,NULL,96337,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,27516.0,2015,27516.0,NULL,NULL,NULL,NULL
10843547,NULL,NULL,NULL,0.0,0.0,5,NULL,NULL,NULL,NULL,73026.0,NULL,NULL,73026,NULL,NULL,6037,NULL,NULL,NULL,NULL,NULL,NULL,33989359,-118394633,63085.0,NULL,NULL,NULL,NULL,NULL,1200,47,LAC2,6.0377030012017E7,51617,3101,NULL,96095,0.0,NULL,NULL,NULL,2,NULL,NULL,1959.0,1,NULL,660680.0,1434941.0,2016,774261.0,20800.37,NULL,NULL,NULL
10859147,NULL,NULL,NULL,0.0,0.0,3,6,NULL,NULL,NULL,5068.0,NULL,NULL,5068,NULL,NULL,6037,NULL,NULL,NULL,NULL,NULL,NULL,34148863,-118437206,7521.0,NULL,NULL,NULL,NULL,NULL,1200,47,LAC2,6.0371412023001E7,12447,3101,27080,96424,0.0,NULL,NULL,NULL,NULL,NULL,NULL,1948.0,1,NULL,580059.0,1174475.0,2016,594416.0,14557.57,NULL,NULL,NULL
10879947,NULL,NULL,NULL,0.0,0.0,4,NULL,NULL,NULL,NULL,1776.0,NULL,NULL,1776,NULL,NULL,6037,NULL,NULL,NULL,NULL,NULL,NULL,34194168,-118385816,8512.0,NULL,NULL,NULL,NULL,NULL,1210,31,LAM1,6.0371232052003E7,12447,3101,46795,96450,0.0,NULL,NULL,NULL,1,NULL,NULL,1947.0,1,NULL,196751.0,440101.0,2016,243350.0,5725.17,NULL,NULL,NULL


In [6]:
train_2016.show(5)

+--------+--------+---------------+
|parcelid|logerror|transactiondate|
+--------+--------+---------------+
|11016594|  0.0276|     2016-01-01|
|14366692| -0.1684|     2016-01-01|
|12098116|  -0.004|     2016-01-01|
|12643413|  0.0218|     2016-01-02|
|14432541|  -0.005|     2016-01-02|
+--------+--------+---------------+
only showing top 5 rows



In [7]:
train_2017.show(5)

+--------+----------------+---------------+
|parcelid|        logerror|transactiondate|
+--------+----------------+---------------+
|14297519| 0.0255949017584|     2017-01-01|
|17052889| 0.0556190874015|     2017-01-01|
|14186244|0.00538285304689|     2017-01-01|
|12177905| -0.103409666332|     2017-01-01|
|10887214|0.00694035920822|     2017-01-01|
+--------+----------------+---------------+
only showing top 5 rows



In [4]:
prop_2016 = prop_2016.withColumnRenamed("parcelid", "parcelIId")
prop_2017 = prop_2017.withColumnRenamed("parcelid", "parcelIId")

In [5]:
df_2016 = train_2016.join(prop_2016, train_2016["parcelid"] == prop_2016["parcelIId"], how="left")
df_2017 = train_2017.join(prop_2017, train_2017["parcelid"] == prop_2017["parcelIId"], how="left")

In [6]:
# rename
for col_name in df_2016.columns: df_2016 = df_2016.withColumnRenamed(col_name, f"{col_name}_16")
for col_name in df_2017.columns: df_2017 = df_2017.withColumnRenamed(col_name, f"{col_name}_17")

In [7]:
# Delete one column in two column parcelid
df_2016 = df_2016.drop('parcelIId_16')
df_2017 = df_2017.drop('parcelIId_17')

In [8]:
df = df_2016.join(df_2017, df_2016["parcelid_16"] == df_2017["parcelid_17"], how="left")

In [9]:
df = df.drop('parcelid_17')
df = df.withColumnRenamed('parcelid_16', 'parcelid')

In [14]:
df.limit(5)

parcelid,logerror_16,transactiondate_16,airconditioningtypeid_16,architecturalstyletypeid_16,basementsqft_16,bathroomcnt_16,bedroomcnt_16,buildingclasstypeid_16,buildingqualitytypeid_16,calculatedbathnbr_16,decktypeid_16,finishedfloor1squarefeet_16,calculatedfinishedsquarefeet_16,finishedsquarefeet12_16,finishedsquarefeet13_16,finishedsquarefeet15_16,finishedsquarefeet50_16,finishedsquarefeet6_16,fips_16,fireplacecnt_16,fullbathcnt_16,garagecarcnt_16,garagetotalsqft_16,hashottuborspa_16,heatingorsystemtypeid_16,latitude_16,longitude_16,lotsizesquarefeet_16,poolcnt_16,poolsizesum_16,pooltypeid10_16,pooltypeid2_16,pooltypeid7_16,propertycountylandusecode_16,propertylandusetypeid_16,propertyzoningdesc_16,rawcensustractandblock_16,regionidcity_16,regionidcounty_16,regionidneighborhood_16,regionidzip_16,roomcnt_16,storytypeid_16,threequarterbathnbr_16,typeconstructiontypeid_16,unitcnt_16,yardbuildingsqft17_16,yardbuildingsqft26_16,yearbuilt_16,numberofstories_16,fireplaceflag_16,structuretaxvaluedollarcnt_16,taxvaluedollarcnt_16,assessmentyear_16,landtaxvaluedollarcnt_16,taxamount_16,taxdelinquencyflag_16,taxdelinquencyyear_16,censustractandblock_16,logerror_17,transactiondate_17,airconditioningtypeid_17,architecturalstyletypeid_17,basementsqft_17,bathroomcnt_17,bedroomcnt_17,buildingclasstypeid_17,buildingqualitytypeid_17,calculatedbathnbr_17,decktypeid_17,finishedfloor1squarefeet_17,calculatedfinishedsquarefeet_17,finishedsquarefeet12_17,finishedsquarefeet13_17,finishedsquarefeet15_17,finishedsquarefeet50_17,finishedsquarefeet6_17,fips_17,fireplacecnt_17,fullbathcnt_17,garagecarcnt_17,garagetotalsqft_17,hashottuborspa_17,heatingorsystemtypeid_17,latitude_17,longitude_17,lotsizesquarefeet_17,poolcnt_17,poolsizesum_17,pooltypeid10_17,pooltypeid2_17,pooltypeid7_17,propertycountylandusecode_17,propertylandusetypeid_17,propertyzoningdesc_17,rawcensustractandblock_17,regionidcity_17,regionidcounty_17,regionidneighborhood_17,regionidzip_17,roomcnt_17,storytypeid_17,threequarterbathnbr_17,typeconstructiontypeid_17,unitcnt_17,yardbuildingsqft17_17,yardbuildingsqft26_17,yearbuilt_17,numberofstories_17,fireplaceflag_17,structuretaxvaluedollarcnt_17,taxvaluedollarcnt_17,assessmentyear_17,landtaxvaluedollarcnt_17,taxamount_17,taxdelinquencyflag_17,taxdelinquencyyear_17,censustractandblock_17
12643413,0.0218,2016-01-02,1,NULL,NULL,2.0,2.0,NULL,4,2.0,NULL,NULL,839.0,839,NULL,NULL,NULL,NULL,6037,NULL,2,NULL,NULL,NULL,2,33755800,-118309000,70859.0,NULL,NULL,NULL,NULL,NULL,010C,266,LAR3,6.0372963002002E7,12447,3101,54300,96222,0.0,NULL,NULL,NULL,1,NULL,NULL,1987.0,NULL,NULL,171518.0,244880.0,2015,73362.0,3048.74,NULL,NULL,60372963002002,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
12098116,-0.004,2016-01-01,1,NULL,NULL,3.0,2.0,NULL,4,3.0,NULL,NULL,2217.0,2217,NULL,NULL,NULL,NULL,6037,NULL,3,NULL,NULL,NULL,2,34136312,-118175032,11423.0,NULL,NULL,NULL,NULL,NULL,0100,261,PSR6,6.0374638003004E7,47019,3101,275411,96293,0.0,NULL,NULL,NULL,1,NULL,NULL,1940.0,NULL,NULL,61994.0,119906.0,2015,57912.0,11484.48,NULL,NULL,60374638003004,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
14366692,-0.1684,2016-01-01,NULL,NULL,NULL,3.5,4.0,NULL,NULL,3.5,NULL,NULL,2263.0,2263,NULL,NULL,NULL,NULL,6059,NULL,3,2,468,NULL,NULL,33668120,-117677556,3643.0,NULL,NULL,NULL,NULL,NULL,1,261,NULL,6.0590524222024E7,32380,1286,NULL,96962,0.0,NULL,1,NULL,NULL,NULL,NULL,2014.0,NULL,NULL,346458.0,585529.0,2015,239071.0,10153.02,NULL,NULL,NULL,0.00214159242517,2017-08-04,NULL,NULL,NULL,3.5,4.0,NUL

In [15]:
print(f"Dataset current have {df.count()} records and {len(df.columns)} columns")

Dataset current have 90277 records and 119 columns


In [16]:
df.dtypes

[('parcelid', 'int'),
 ('logerror_16', 'double'),
 ('transactiondate_16', 'date'),
 ('airconditioningtypeid_16', 'int'),
 ('architecturalstyletypeid_16', 'int'),
 ('basementsqft_16', 'int'),
 ('bathroomcnt_16', 'double'),
 ('bedroomcnt_16', 'double'),
 ('buildingclasstypeid_16', 'int'),
 ('buildingqualitytypeid_16', 'int'),
 ('calculatedbathnbr_16', 'double'),
 ('decktypeid_16', 'int'),
 ('finishedfloor1squarefeet_16', 'int'),
 ('calculatedfinishedsquarefeet_16', 'double'),
 ('finishedsquarefeet12_16', 'int'),
 ('finishedsquarefeet13_16', 'int'),
 ('finishedsquarefeet15_16', 'int'),
 ('finishedsquarefeet50_16', 'int'),
 ('finishedsquarefeet6_16', 'int'),
 ('fips_16', 'int'),
 ('fireplacecnt_16', 'int'),
 ('fullbathcnt_16', 'int'),
 ('garagecarcnt_16', 'int'),
 ('garagetotalsqft_16', 'int'),
 ('hashottuborspa_16', 'boolean'),
 ('heatingorsystemtypeid_16', 'int'),
 ('latitude_16', 'int'),
 ('longitude_16', 'int'),
 ('lotsizesquarefeet_16', 'double'),
 ('poolcnt_16', 'int'),
 ('poolsizesu

In [10]:
# quantitative_variables
quantitative_vars = [col[0] for col in df.dtypes if col[1] in ['int', 'bigint', 'double']]

In [11]:
# Qualitative_variables
Qualitative_vars = [col[0] for col in df.dtypes if col[1] not in ['int', 'bigint', 'double']]

In [19]:
df.select(quantitative_vars).limit(10)

parcelid,logerror_16,airconditioningtypeid_16,architecturalstyletypeid_16,basementsqft_16,bathroomcnt_16,bedroomcnt_16,buildingclasstypeid_16,buildingqualitytypeid_16,calculatedbathnbr_16,decktypeid_16,finishedfloor1squarefeet_16,calculatedfinishedsquarefeet_16,finishedsquarefeet12_16,finishedsquarefeet13_16,finishedsquarefeet15_16,finishedsquarefeet50_16,finishedsquarefeet6_16,fips_16,fireplacecnt_16,fullbathcnt_16,garagecarcnt_16,garagetotalsqft_16,heatingorsystemtypeid_16,latitude_16,longitude_16,lotsizesquarefeet_16,poolcnt_16,poolsizesum_16,pooltypeid10_16,pooltypeid2_16,pooltypeid7_16,propertylandusetypeid_16,rawcensustractandblock_16,regionidcity_16,regionidcounty_16,regionidneighborhood_16,regionidzip_16,roomcnt_16,storytypeid_16,threequarterbathnbr_16,typeconstructiontypeid_16,unitcnt_16,yardbuildingsqft17_16,yardbuildingsqft26_16,yearbuilt_16,numberofstories_16,structuretaxvaluedollarcnt_16,taxvaluedollarcnt_16,assessmentyear_16,landtaxvaluedollarcnt_16,taxamount_16,taxdelinquencyyear_16,censustractandblock_16,logerror_17,airconditioningtypeid_17,architecturalstyletypeid_17,basementsqft_17,bathroomcnt_17,bedroomcnt_17,buildingclasstypeid_17,buildingqualitytypeid_17,calculatedbathnbr_17,decktypeid_17,finishedfloor1squarefeet_17,calculatedfinishedsquarefeet_17,finishedsquarefeet12_17,finishedsquarefeet13_17,finishedsquarefeet15_17,finishedsquarefeet50_17,finishedsquarefeet6_17,fips_17,fireplacecnt_17,fullbathcnt_17,garagecarcnt_17,garagetotalsqft_17,heatingorsystemtypeid_17,latitude_17,longitude_17,lotsizesquarefeet_17,poolcnt_17,poolsizesum_17,pooltypeid10_17,pooltypeid2_17,pooltypeid7_17,propertylandusetypeid_17,rawcensustractandblock_17,regionidcity_17,regionidcounty_17,regionidneighborhood_17,regionidzip_17,roomcnt_17,storytypeid_17,threequarterbathnbr_17,typeconstructiontypeid_17,unitcnt_17,yardbuildingsqft17_17,yardbuildingsqft26_17,yearbuilt_17,numberofstories_17,structuretaxvaluedollarcnt_17,taxvaluedollarcnt_17,assessmentyear_17,landtaxvaluedollarcnt_17,taxamount_17,taxdelinquencyyear_17,censustractandblock_17
12286022,0.044,NULL,NULL,NULL,1.0,2.0,NULL,7,1.0,NULL,NULL,1297.0,1297,NULL,NULL,NULL,NULL,6037,NULL,1,NULL,NULL,7,33899475,-118212720,6677.0,NULL,NULL,NULL,NULL,NULL,261,6.0375416053007E7,24174,3101,NULL,96091,0.0,NULL,NULL,NULL,1,NULL,NULL,1939.0,NULL,64549.0,111521.0,2015,46972.0,2304.97,NULL,60375416053007,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
12643413,0.0218,1,NULL,NULL,2.0,2.0,NULL,4,2.0,NULL,NULL,839.0,839,NULL,NULL,NULL,NULL,6037,NULL,2,NULL,NULL,2,33755800,-118309000,70859.0,NULL,NULL,NULL,NULL,NULL,266,6.0372963002002E7,12447,3101,54300,96222,0.0,NULL,NULL,NULL,1,NULL,NULL,1987.0,NULL,171518.0,244880.0,2015,73362.0,3048.74,NULL,60372963002002,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
14677559,0.0843,NULL,NULL,NULL,2.0,2.0,NULL,NULL,2.0,NULL,NULL,1260.0,1260,NULL,NULL,NULL,NULL,6059,NULL,2,1,0,NULL,33612700,-117742000,NULL,NULL,NULL,NULL,NULL,NULL,266,6.0590626483005E7,46098,1286,NULL,96963,5.0,NULL,NULL,NULL,NULL,NULL,NULL,1977.0,1,109977.0,190960.0,2015,80983.0,1940.26,NULL,60590626483015,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
12098116,-0.004,1,NULL,NULL,3.0,2.0,NULL,4,3.0,NULL,NULL,2217.0,2217,NULL,NULL,NULL,NULL,6037,NULL,3,NULL,NULL,2,34136312,-118175032,11423.0,NULL,NULL,NULL,NULL,NULL,261,6.0374638003004E7,47019,3101,275411,9

In [20]:
df.select(Qualitative_vars).limit(10)

transactiondate_16,hashottuborspa_16,propertycountylandusecode_16,propertyzoningdesc_16,fireplaceflag_16,taxdelinquencyflag_16,transactiondate_17,hashottuborspa_17,propertycountylandusecode_17,propertyzoningdesc_17,fireplaceflag_17,taxdelinquencyflag_17
2016-01-02,NULL,0100,CORH*,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2016-01-02,NULL,010C,LAR3,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2016-01-03,NULL,34,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2016-01-01,NULL,0100,PSR6,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2016-01-01,NULL,1,NULL,NULL,NULL,2017-08-04,NULL,1,NULL,NULL,NULL
2016-01-01,NULL,0100,LARS,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2016-01-02,NULL,0100,HBR3YY,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2016-01-02,NULL,122,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2016-01-02,NULL,1129,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2016-01-02,NULL,34,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [21]:
df.select(quantitative_vars).describe().limit(10)

In [22]:
df.select(Qualitative_vars).describe().limit(10)

summary,propertycountylandusecode_16,propertyzoningdesc_16,taxdelinquencyflag_16,propertycountylandusecode_17,propertyzoningdesc_17,taxdelinquencyflag_17
count,90276,58314,1783,2359,1618,132
mean,197.03031493485236,NULL,NULL,174.68393030009682,NULL,NULL
stddev,304.6516248942232,NULL,NULL,266.9103545007249,NULL,NULL
min,0,1NR1*,Y,0100,AHR17000*,Y
max,96,WVRR1-RPD1,Y,96,WVRPD18U*,Y


In [23]:
for col in quantitative_vars:
    print(f"{col}: {df.filter(df[col].isNull()).count()}")

parcelid: 0
logerror_16: 0
airconditioningtypeid_16: 61495
architecturalstyletypeid_16: 90016
basementsqft_16: 90234
bathroomcnt_16: 0
bedroomcnt_16: 0
buildingclasstypeid_16: 90261
buildingqualitytypeid_16: 32912
calculatedbathnbr_16: 1182
decktypeid_16: 89619
finishedfloor1squarefeet_16: 83421
calculatedfinishedsquarefeet_16: 661
finishedsquarefeet12_16: 4679
finishedsquarefeet13_16: 90244
finishedsquarefeet15_16: 86713
finishedsquarefeet50_16: 83421
finishedsquarefeet6_16: 89856
fips_16: 0
fireplacecnt_16: 80670
fullbathcnt_16: 1182
garagecarcnt_16: 60339
garagetotalsqft_16: 60339
heatingorsystemtypeid_16: 34196
latitude_16: 0
longitude_16: 0
lotsizesquarefeet_16: 10150
poolcnt_16: 72376
poolsizesum_16: 89308
pooltypeid10_16: 89116
pooltypeid2_16: 89073
pooltypeid7_16: 73580
propertylandusetypeid_16: 0
rawcensustractandblock_16: 0
regionidcity_16: 1803
regionidcounty_16: 0
regionidneighborhood_16: 54265
regionidzip_16: 35
roomcnt_16: 0
storytypeid_16: 90234
threequarterbathnbr_16: 7

In [24]:
for col in Qualitative_vars:
    print(f"{col}: {df.filter(df[col].isNull()).count()}")

transactiondate_16: 0
hashottuborspa_16: 87912
propertycountylandusecode_16: 1
propertyzoningdesc_16: 31963
fireplaceflag_16: 90055
taxdelinquencyflag_16: 88494
transactiondate_17: 87918
hashottuborspa_17: 90242
propertycountylandusecode_17: 87918
propertyzoningdesc_17: 88659
fireplaceflag_17: 90270
taxdelinquencyflag_17: 90145


In [12]:
# Drop columns that are more than 50% missing
def column_dropper(df, threshold):
    total_records = df.count()
    for col in df.columns:
        missing = df.filter(df[col].isNull()).count()
        missing_percent = missing / total_records
        if missing_percent > threshold:
            df = df.drop(col)
    return df

df = column_dropper(df, 0.5)

In [14]:
# fillna for quantitative_variables
for column in df.columns:
    if column in quantitative_vars:
        df = df.fillna({column: df.select(avg(column)).collect()[0][0]})

In [16]:
# fillna for Qualitative_variables
for column in df.columns:
    if column in Qualitative_vars:
        if column == "transactiondate_16":
            df = df.withColumn("transactiondate_16", col("transactiondate_16").cast("string"))
            avg_value = df.select(mode(column)).collect()[0][0]
            df.na.fill({column: avg_value})
            df = df.withColumn("transactiondate_16", col("transactiondate_16").cast("date"))
        else:
            avg_value = df.select(mode(column)).collect()[0][0]
            df.na.fill({column: avg_value})

AssertionError: Undefined error message parameter for error class: CANNOT_PARSE_DATATYPE. Parameters: {'error': '[WinError 10061] No connection could be made because the target machine actively refused it'}

In [14]:
for col in df.columns:
    print(f"{col}: {df.filter(df[col].isNull()).count()}")

parcelid: 0
logerror_16: 0
transactiondate_16: 0
bathroomcnt_16: 0
bedroomcnt_16: 0
buildingqualitytypeid_16: 32912
calculatedbathnbr_16: 1182
calculatedfinishedsquarefeet_16: 661
finishedsquarefeet12_16: 4679
fips_16: 0


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "C:\Users\MSI Modern 14\AppData\Local\Programs\Python\Python311\Lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MSI Modern 14\AppData\Local\Programs\Python\Python311\Lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MSI Modern 14\AppData\Local\Programs\Python\Python311\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [40]:
df.printSchema()

In [ ]:
df.count(), len(df.columns)

# Data Visualization